In [10]:
import pandas as pd
df = pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [11]:
df.shape

(768, 9)

In [2]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [3]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [57]:
from scipy import stats
min(stats.zscore(df.DiabetesPedigreeFunction,axis = 0))

-1.1895531764897842

In [23]:
df1 = df[df.Pregnancies<=df.Pregnancies.mean()+3*df.Pregnancies.std()]
df1.shape

(764, 9)

In [24]:
df2 = df1[df1.Glucose>df1.Glucose.mean()-3*df1.Glucose.std()]
df2.shape

(759, 9)

In [25]:
df3 = df2[df2.BloodPressure>df2.BloodPressure.mean()-3*df2.BloodPressure.std()]
df3.shape

(724, 9)

In [29]:
df4 = df3[df3.SkinThickness<=df3.SkinThickness.mean()+3*df3.SkinThickness.std()]
df4.shape

(723, 9)

In [44]:
df5 = df4[df4.Insulin<=df4.Insulin.mean()+3*df4.Insulin.std()]
df5.shape

(705, 9)

In [47]:
df6 = df5[df5.BMI<=df5.BMI.mean()+3*df5.BMI.std()]
df7 = df6[df6.BMI>df6.BMI.mean()-3*df6.BMI.std()]
df7.shape

(697, 9)

In [50]:
df8 = df7[df7.DiabetesPedigreeFunction<=df7.DiabetesPedigreeFunction.mean()+3*df7.DiabetesPedigreeFunction.std()]
df8.shape

(688, 9)

In [51]:
df8.sample(3)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
255,1,113,64,35,0,33.6,0.543,21,1
668,6,98,58,33,190,34.0,0.430,43,0
558,11,103,68,40,0,46.2,0.126,42,0


In [55]:
x = df8.drop('Outcome',axis = 1)
y = df8.Outcome

In [58]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x)

In [59]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X_scaled,y,stratify = y,random_state = 20)

In [61]:
y_train.value_counts()

0    346
1    170
Name: Outcome, dtype: int64

In [85]:
from sklearn.svm import SVC 
model = SVC(C = 10,gamma = 0.1,kernel = "linear")
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.7325581395348837

In [86]:
from sklearn.ensemble import BaggingClassifier
Bag_model = BaggingClassifier(
                    base_estimator = SVC(C = 10,gamma = 0.1,kernel = "linear"),
                    n_estimators = 100,
                    max_samples = 0.8,
                    oob_score = True,
                    random_state = 0
                    )
Bag_model.fit(x_train,y_train)
Bag_model.oob_score_

0.7887596899224806

In [87]:
Bag_model.score(x_test,y_test)

0.7325581395348837

In [80]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.6802325581395349

In [84]:
from sklearn.ensemble import BaggingClassifier
Bag_model = BaggingClassifier(
                    base_estimator = DecisionTreeClassifier(),
                    n_estimators = 100,
                    max_samples = 0.8,
                    oob_score = True,
                    random_state = 0
                    )
Bag_model.fit(x_train,y_train)
Bag_model.oob_score_

0.7848837209302325

In [83]:
Bag_model.score(x_test,y_test)

0.75